In [53]:
import gym
import gym_jumping_task
from gym_jumping_task.envs.jumping_task import JumpTaskEnv
import matplotlib.pyplot as plt
import numpy as np
from torchvision.utils import make_grid
from env import VanillaEnv
from rl.common.buffer2 import Transition, Episode, RolloutBuffer
from rl.ppo.policies import ActorCriticNet
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from typing import List
from rl.ppo.ppo import PPO

In [54]:
class BCDataset(Dataset):
    def __init__(self, x, y):
        super().__init__()
        assert x.shape[0] == y.shape[0]
        self.x = x
        self.y = y

    def __len__(self):
        return self.y.shape[0]
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]

In [55]:
def generate_expert_trajectories(env):
    n_iterations = 10_00
    gamma = 0.99

    episodes: List[Episode] = []

    for i in range(n_iterations):
        done = False
        episode = Episode(discount=gamma)
        obs = env.reset()
        obstacle_position = env.actualEnv.obstacle_position;
        jumping_pixel = obstacle_position-14;
        step = 0
        episode_reward = 0
        while not done:
            action = 1 if step==jumping_pixel else 0
            obs, reward, done, _ = env.step(action)
            episode.append(Transition(obs, action, reward, 0))
            env.render()
            step +=1
        episodes.append(episode)
        if i % 100==0: print(f"Episode {i} return {np.sum(episode.rewards())}")

    # get the states, returns and actions out of the episodes
    states, actions, values = [], [], []
    for episode in episodes:
        states += episode.states()
        actions += episode.actions()
        values += episode.calculate_return()

    states, actions, values = np.array(states), np.array(actions), np.array(values)
    # vertically add actions and values
    targets = np.column_stack((actions, values))

    X = torch.tensor(states)
    Y = torch.tensor(targets)
    data: BCDataset = BCDataset(X, Y)
    train_set, val_set = torch.utils.data.random_split(data, [int(len(data)*0.8), int(len(data)*0.2)])

    trainloader:DataLoader = DataLoader(train_set, batch_size=64, shuffle=True)
    testloader:DataLoader = DataLoader(val_set, batch_size=64, shuffle=True)
    return trainloader, testloader


In [56]:
@torch.enable_grad()
def train(net:ActorCriticNet, dataLoader: DataLoader, optim_actor, optim_critic, loss_actor, loss_critic) -> tuple:
    device = next(net.parameters()).device
    net.train()
    actor_errors, critic_errors = [], []

    for batch in dataLoader:
        # X is the observation
        # y contains the choosen action and the return estimate from the critic
        X, y = batch[0].to(device), batch[1].to(device)
        target_actions = y[:,0]
        target_returns = y[:,1]

        # normalize returns
        target_returns = (target_returns - target_returns.mean()) / (target_returns.std() + 1e-5)

        pred_action_logits = net.actor.forward(X)
        pred_values = net.critic.forward(X).squeeze()

        actor_error = loss_actor(pred_action_logits, target_actions.type(torch.LongTensor).to(device))
        critic_error = loss_critic(pred_values, target_returns.type(torch.FloatTensor).to(device))

        optim_actor.zero_grad()
        actor_error.backward()
        optim_actor.step()

        optim_critic.zero_grad()
        critic_error.backward()
        optim_critic.step()
        
        actor_errors.append(actor_error.item())
        critic_errors.append(critic_error.item())

    return actor_errors, critic_errors

@torch.no_grad()
def evaluate(net:ActorCriticNet, dataLoader: DataLoader, loss_actor, loss_critic) -> tuple:
    device = next(net.parameters()).device
    net.eval()
    actor_errors, critic_errors = [], []

    for batch in dataLoader:
        X, y = batch[0].to(device), batch[1].to(device)
        target_actions = y[:,0]
        target_returns = y[:,1]

        # normalize returns
        target_returns = (target_returns - target_returns.mean()) / (target_returns.std() + 1e-5)

        pred_action_logits = net.actor.forward(X)
        pred_values = net.critic.forward(X).squeeze()

        actor_errors.append(loss_actor(pred_action_logits, target_actions.type(torch.LongTensor).to(device)).item())
        critic_errors.append(loss_critic(pred_values, target_returns.type(torch.FloatTensor).to(device)).item())

    return actor_errors, critic_errors

In [57]:
train_conf = {
    "narrow_grid": {
        # (obstacle_pos, floor_height)
        (26, 12), (29, 12), (31, 12), (34, 12),
        (26, 20), (29, 20), (31, 20), (34, 20),
        (26, 28), (29, 28), (31, 28), (34, 28),
    },
    "wide_grid": {
        # (obstacle_pos, floor_height)
        (22, 8), (27, 8), (32, 8), (38, 8),
        (22, 20), (27, 20), (32, 20), (38, 20),
        (22, 32), (27, 32), (32, 32), (38, 32),
    },
    "random_grid": {
        # (obstacle_pos, floor_height)
        (15, 36), (17, 8), (19, 20), (21, 32),
        (26, 20), (30, 4), (32, 24), (34, 36),
        (36, 4), (38, 16), (43, 12), (44, 28),
    },
    "diagonal_grid": {
        (17, 8), (21, 12), (25, 16), (29, 20),
        (32, 24), (36, 28), (40, 32), (44, 36),
    },
}

In [59]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Training on ", device)

lr = 0.004
n_epochs = 5
for conf_name in list(train_conf.keys()):
    env = VanillaEnv(configurations=list(train_conf[conf_name]),rendering=True)

    trainloader, testloader = generate_expert_trajectories(env)
    model = ActorCriticNet().to(device);
    optim_actor = optim.Adam(model.actor.parameters(), lr=lr);
    optim_critic = optim.Adam(model.critic.parameters(), lr=lr)
    loss_actor = nn.CrossEntropyLoss()
    loss_critic = nn.MSELoss()

    for epoch in range(n_epochs):
        actor_errors, critic_errors = train(model, trainloader, optim_actor, optim_critic, loss_actor, loss_critic)
        val_actor_errors, val_critic_errors = evaluate(model, testloader, loss_actor, loss_critic)
        print(f"""Epoch {epoch} Train errors: Actor {np.mean(actor_errors):.3f}, Critic {np.mean(actor_errors):.3f} 
              Val errors: Actor {np.mean(val_actor_errors):.3f}, Critic {np.mean(val_critic_errors):.3f} """)

        optimizer = optim.Adam(model.parameters(), lr=0.001)
        ppo = PPO(model, env, optimizer)

        ppo.save('./ckpts-final', 'BC-' + conf_name, info={'conf': list(train_conf[conf_name])})

Training on  cuda
Episode 0 return 156.0
Episode 100 return 156.0
Episode 200 return 156.0
Episode 300 return 156.0
Episode 400 return 156.0
Episode 500 return 156.0
Episode 600 return 156.0
Episode 700 return 156.0
Episode 800 return 156.0
Episode 900 return 156.0
Epoch 0 Train errors: Actor 0.016, Critic 0.016 
              Val errors: Actor 0.000, Critic 0.023 
Epoch 1 Train errors: Actor 0.000, Critic 0.000 
              Val errors: Actor 0.000, Critic 0.026 
Epoch 2 Train errors: Actor 0.000, Critic 0.000 
              Val errors: Actor 0.000, Critic 0.026 
Epoch 3 Train errors: Actor 0.000, Critic 0.000 
              Val errors: Actor 0.000, Critic 0.024 
Epoch 4 Train errors: Actor 0.000, Critic 0.000 
              Val errors: Actor 0.000, Critic 0.053 
Episode 0 return 156.0
Episode 100 return 156.0
Episode 200 return 156.0
Episode 300 return 156.0
Episode 400 return 156.0
Episode 500 return 156.0
Episode 600 return 156.0
Episode 700 return 156.0
Episode 800 return 156.0
E